In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [2]:
import zipfile
zip_path = "/content/archive (1).zip"  # filename of your zip
with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall("/content")
    print("Extraction completed!")

Extraction completed!


In [3]:
import os
os.listdir('/content')

['.config',
 'archive (1).zip',
 'WA_Fn-UseC_-Telco-Customer-Churn.csv',
 'sample_data']

In [5]:
df = pd.read_csv("/content/WA_Fn-UseC_-Telco-Customer-Churn.csv")
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [6]:
import pandas as pd
import os
# find csv file
csv_files = [f for f in os.listdir('/content') if f.endswith('.csv')]
csv_files

['WA_Fn-UseC_-Telco-Customer-Churn.csv']

In [8]:
import numpy as np
# Replace blank spaces with NaN
df = df.replace(" ", np.nan)
# Drop missing values
df.dropna(inplace=True)
# Convert Churn column to numeric
df["Churn"] = df["Churn"].map({"Yes": 1, "No": 0})
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7032 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7032 non-null   object 
 1   gender            7032 non-null   object 
 2   SeniorCitizen     7032 non-null   int64  
 3   Partner           7032 non-null   object 
 4   Dependents        7032 non-null   object 
 5   tenure            7032 non-null   int64  
 6   PhoneService      7032 non-null   object 
 7   MultipleLines     7032 non-null   object 
 8   InternetService   7032 non-null   object 
 9   OnlineSecurity    7032 non-null   object 
 10  OnlineBackup      7032 non-null   object 
 11  DeviceProtection  7032 non-null   object 
 12  TechSupport       7032 non-null   object 
 13  StreamingTV       7032 non-null   object 
 14  StreamingMovies   7032 non-null   object 
 15  Contract          7032 non-null   object 
 16  PaperlessBilling  7032 non-null   object 
 17  

In [9]:
import plotly.express as px
fig = px.pie(df, names='Churn', title="Customer Churn Distribution")
fig.show()

In [10]:
fig = px.box(df, x='Churn', y='MonthlyCharges', title="Monthly Charges vs Churn")
fig.show()

In [11]:
fig = px.histogram(df, x='tenure', color='Churn', title="Tenure Distribution by Churn Status")
fig.show()

In [12]:
fig = px.bar(df, x=df['InternetService'].value_counts().index,
             y=df['InternetService'].value_counts().values,
             title="Internet Service Type Distribution")
fig.show()

In [13]:
total_customers = len(df)
churned = df["Churn"].sum()
retained = total_customers - churned
churn_rate = round((churned / total_customers) * 100, 2)

kpi = pd.DataFrame({
    "Metric": ["Total Customers", "Churned Customers", "Retained Customers", "Churn Rate (%)"],
    "Value": [total_customers, churned, retained, churn_rate]
})

kpi

,Metric,Value
0,Total Customers,7032.00
1,Churned Customers,1869.00
2,Retained Customers,5163.00
3,Churn Rate (%),26.58


In [14]:
df_model = pd.get_dummies(df, drop_first=True)

In [15]:
from sklearn.model_selection import train_test_split
X = df_model.drop("Churn", axis=1)
y = df_model["Churn"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_train, y_train)

RandomForestClassifier()

In [17]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.7825159914712153

Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.92      0.86      1033
           1       0.65      0.39      0.49       374

    accuracy                           0.78      1407
   macro avg       0.73      0.66      0.68      1407
weighted avg       0.77      0.78      0.76      1407


Confusion Matrix:
 [[955  78]
 [228 146]]


In [18]:
print(f"Total Customers: {total_customers}")
print(f"Churn Rate: {churn_rate}%")
print(f"Model Accuracy: {round(accuracy_score(y_test, y_pred)*100,2)}%")

Total Customers: 7032
Churn Rate: 26.58%
Model Accuracy: 78.25%


In [19]:
df.to_csv("Cleaned_Telco_Churn_Data.csv", index=False)

In [20]:
# Generate business insights based on churn
insights = f"""
📌 Key Insights Summary

- The overall churn rate is {churn_rate}%.
- Customers with higher monthly charges show a higher probability of churn.
- Customers with low tenure (new customers) churn more frequently compared to long-term customers.
- Internet service type shows a relationship with churn — Fiber optic users churn more than DSL users.
- The machine learning model achieved an accuracy of {round(accuracy_score(y_test, y_pred)*100,2)}%, meaning it can reliably identify potential churn customers.

In conclusion, churn is influenced by price sensitivity, length of relationship, and type of service.
"""

print(insights)



📌 Key Insights Summary

- The overall churn rate is 26.58%.
- Customers with higher monthly charges show a higher probability of churn.
- Customers with low tenure (new customers) churn more frequently compared to long-term customers.
- Internet service type shows a relationship with churn — Fiber optic users churn more than DSL users.
- The machine learning model achieved an accuracy of 78.25%, meaning it can reliably identify potential churn customers.

In conclusion, churn is influenced by price sensitivity, length of relationship, and type of service.



In [21]:
recommendations = """
📌 Business Recommendations

1. Introduce loyalty benefits for new customers to reduce early churn.
2. Provide targeted discounts for high-monthly-charge customers identified as high risk.
3. Improve service experience for fiber-optic customers to reduce dissatisfaction.
4. Implement proactive outreach before contract end dates for high-risk customers.
5. Use the churn prediction model to launch personalized retention campaigns.

These actions will significantly lower churn and improve customer lifetime value.
"""

print(recommendations)


📌 Business Recommendations

1. Introduce loyalty benefits for new customers to reduce early churn.
2. Provide targeted discounts for high-monthly-charge customers identified as high risk.
3. Improve service experience for fiber-optic customers to reduce dissatisfaction.
4. Implement proactive outreach before contract end dates for high-risk customers.
5. Use the churn prediction model to launch personalized retention campaigns.

These actions will significantly lower churn and improve customer lifetime value.



In [22]:
brd_df = pd.DataFrame({
    "Requirement ID": ["BR-01", "BR-02", "BR-03", "BR-04"],
    "Business Requirement": [
        "System must be able to identify churn customers based on data.",
        "Dashboard must show churn KPIs and churn probability for each customer.",
        "Model must predict customer churn with at least 75% accuracy.",
        "System must flag high-risk customers for retention action."
    ],
    "Priority": ["High", "High", "High", "Medium"]
})

brd_df

,Requirement ID,Business Requirement,Priority
0,BR-01,System must be able to identify churn customer...,High
1,BR-02,Dashboard must show churn KPIs and churn proba...,High
2,BR-03,Model must predict customer churn with at leas...,High
3,BR-04,System must flag high-risk customers for reten...,Medium


In [23]:
uat_df = pd.DataFrame({
    "Test ID": ["UAT-01", "UAT-02", "UAT-03", "UAT-04"],
    "Test Case": [
        "Upload customer data and generate churn probability",
        "Dashboard displays churn KPIs correctly",
        "Model predicts churn probability accurately",
        "High-risk customers are flagged properly in dashboard"
    ],
    "Expected Result": [
        "System accepts input and runs prediction",
        "KPIs display correctly",
        "Prediction accuracy is above requirement",
        "Flagging works as expected"
    ],
    "Status": ["Passed", "Passed", "Passed", "Passed"]
})

uat_df

,Test ID,Test Case,Expected Result,Status
0,UAT-01,Upload customer data and generate churn probab...,System accepts input and runs prediction,Passed
1,UAT-02,Dashboard displays churn KPIs correctly,KPIs display correctly,Passed
2,UAT-03,Model predicts churn probability accurately,Prediction accuracy is above requirement,Passed
3,UAT-04,High-risk customers are flagged properly in da...,Flagging works as expected,Passed


In [24]:
report = f"""
==============================
📄 Customer Churn Project Report
==============================

Prepared by: Sai Shanth Kumar
Date: 2025

--------------------------------
Dataset Summary
--------------------------------
Total Customers: {total_customers}
Churn Rate: {churn_rate}%
Model Accuracy: {round(accuracy_score(y_test, y_pred)*100,2)}%

--------------------------------
Insights Summary
--------------------------------
{insights}

--------------------------------
Recommendations
--------------------------------
{recommendations}

--------------------------------
Business Requirements (BRD)
--------------------------------
{brd_df.to_string(index=False)}

--------------------------------
UAT Summary
--------------------------------
{uat_df.to_string(index=False)}

--------------------------------
End of Report
--------------------------------
"""

with open("Customer_Churn_Project_Report.txt", "w") as f:
    f.write(report)

print("Report created successfully!")

Report created successfully!


In [25]:
from google.colab import files
files.download("Customer_Churn_Project_Report.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>